# Implementacija modela za prepoznavanje broja prstiju na ruci pomoću konvolucijskih neuronskih mreža
Kolegij: Neuronske mreže i duboko učenje
Autor: Krešimir Špehar (0165079422)

In [ ]:
#Povezivanje slika na google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importanje biblioteka

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import os
import numpy as np


In [ ]:
target_size = (300, 300)  #Veličina slika
num_classes = 5
batch_size = 128          #Batch za treniranje
epochs = 13

data_dir = '/content/drive/MyDrive/project_folder/train'      #Putanja do foldera slika

def load_and_preprocess_images(directory, target_size):       #Učitavanje i obrada slika
    X = []
    y = []
    class_labels = sorted(os.listdir(directory))

    for label_idx, class_label in enumerate(class_labels):
        class_dir = os.path.join(directory, class_label)
        for img_filename in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_filename)
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img)
            img_array /= 255.0
            X.append(img_array)
            y.append(label_idx)

    X = np.array(X)
    y = to_categorical(y, num_classes=num_classes)

    return X, y

X, y = load_and_preprocess_images(data_dir, target_size)          #Pozivanje funkcije iznad te spremanje u X i y

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)         #Podjela slika u training, validation i test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

input_shape = X_train.shape[1:]         #Oblik slike bez veličine batcha

In [ ]:
#Prikaz koliko slika ima po svakoj klasi
class_counts = {}
class_labels = sorted(os.listdir(data_dir))

for class_label in class_labels:
    class_dir = os.path.join(data_dir, class_label)
    num_images = len(os.listdir(class_dir))
    class_counts[class_label] = num_images

for class_label, count in class_counts.items():
    print(f"Class '{class_label}' has {count} images.")


Class 'class_1' has 230 images.
Class 'class_2' has 232 images.
Class 'class_3' has 232 images.
Class 'class_4' has 230 images.
Class 'class_5' has 225 images.


In [ ]:
#Prvi model

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])  #Compile modela

In [ ]:
model.summary()     #Prikaz strukture i parametara modela

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 341056)            0         
                                                                 
 dense (Dense)               (None, 128)               4

In [ ]:
#Prva faza

history = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val),
)

#Evalucija modela na test podacima
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")


In [ ]:
#Drugi model

batch_size = 32
epochs = 15

model = Sequential()

model.add(Conv2D(32, (3, 3), 1, activation='relu', input_shape=(target_size[0], target_size[1], 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (3, 3), 1, activation='relu'))      #Dodani konvolucijski sloj
model.add(MaxPooling2D((2, 2)))                           #Dodani sloj udruživanja

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))                                   #Dodan dropout
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Druga faza

history = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val),

)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/15
29/29 [==============================] - 19s 185ms/step - loss: 2.1345 - accuracy: 0.1948 - val_loss: 1.6176 - val_accuracy: 0.1826
Epoch 2/15
29/29 [==============================] - 3s 107ms/step - loss: 1.6201 - accuracy: 0.2100 - val_loss: 1.6209 - val_accuracy: 0.1478
Epoch 3/15
29/29 [==============================] - 3s 96ms/step - loss: 1.6171 - accuracy: 0.2231 - val_loss: 1.5995 - val_accuracy: 0.2957
Epoch 4/15
29/29 [==============================] - 3s 93ms/step - loss: 1.5976 - accuracy: 0.2731 - val_loss: 1.5604 - val_accuracy: 0.2870
Epoch 5/15
29/29 [==============================] - 3s 94ms/step - loss: 1.5902 - accuracy: 0.2742 - val_loss: 1.5413 - val_accuracy: 0.2870
Epoch 6/15
29/29 [==============================] - 3s 93ms/step - loss: 1.5561 - accuracy: 0.2971 - val_loss: 1.5187 - val_accuracy: 0.3130
Epoch 7/15
29/29 [==============================] - 3s 103ms/step - loss: 1.4764 - accuracy: 0.3526 - val_loss: 1.4595 - val_accuracy: 0.3565
Epoch 8/1

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 128)      0

In [ ]:
#Funkcija za plotiranje training i validation lossa te training i validation accuracya
import matplotlib.pyplot as plt

train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
epochs = range(1, len(train_loss) + 1)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracy, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()